In [1]:
import itertools
import os
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
#import tensorflow_addons as tfa
 
print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

C:\Users\Chris\anaconda3\envs\tfh1\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


TF version: 2.6.0
Hub version: 0.8.0
GPU is available


In [2]:
model_name = "efficientnetv2-s-21k" 
model_handle_map = {
  "efficientnetv2-s-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/feature_vector/2",
  "efficientnetv2-m-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_m/feature_vector/2",
  "efficientnetv2-l-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_l/feature_vector/2",
  "efficientnetv2-xl-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_xl/feature_vector/2",
  "efficientnetv2-b0-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b0/feature_vector/2",
  "efficientnetv2-b1-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b1/feature_vector/2",
  "efficientnetv2-b2-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b2/feature_vector/2",
  "efficientnetv2-b3-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b3/feature_vector/2",
  "swin_s3_base_224": "https://tfhub.dev/sayannath/mobilevit_s_1k_256/1",
}

model_image_size_map = {
  "efficientnetv2-s-21k": 384,
  "efficientnetv2-m-21k": 480,
  "efficientnetv2-l-21k": 480,
  "efficientnetv2-xl-21k": 512,
  "efficientnetv2-b0-21k": 224,
  "efficientnetv2-b1-21k": 240,
  "efficientnetv2-b2-21k": 260,
  "efficientnetv2-b3-21k": 300,
  "swin_s3_base_224": 224, 
}

model_handle = model_handle_map.get(model_name)
pixels = model_image_size_map.get(model_name, 224)
print(f"Selected model: {model_name} : {model_handle}")
IMAGE_SIZE = (pixels, pixels)
print(f"Input size {IMAGE_SIZE}")
BATCH_SIZE = 12

Selected model: efficientnetv2-s-21k : https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/feature_vector/2
Input size (384, 384)


In [3]:
data_dir = ("./data/")
def build_dataset(subset):
  return tf.keras.preprocessing.image_dataset_from_directory(
      data_dir,
      validation_split=.20,
      subset=subset,
      label_mode="categorical",
      shuffle = True,
      seed=1,
      # Seed needs to provided when using validation_split and shuffle = True.
      # A fixed seed is used so that the validation set is stable across runs.
      image_size=IMAGE_SIZE,
      batch_size=1)

train_ds = build_dataset("training")
class_names = tuple(train_ds.class_names)
train_size = train_ds.cardinality().numpy()
train_ds = train_ds.unbatch().batch(BATCH_SIZE)
train_ds = train_ds.repeat()

normalization_layer = tf.keras.layers.Rescaling(1. / 255)
preprocessing_model = tf.keras.Sequential([normalization_layer])
do_data_augmentation = True
if do_data_augmentation:
    preprocessing_model.add(
        tf.keras.layers.RandomFlip(mode="horizontal_and_vertical"))

train_ds = train_ds.map(lambda images, labels:
                        (preprocessing_model(images), labels))

val_ds = build_dataset("validation")
valid_size = val_ds.cardinality().numpy()
val_ds = val_ds.unbatch().batch(BATCH_SIZE)
val_ds = val_ds.map(lambda images, labels:
                    (normalization_layer(images), labels))

Found 89514 files belonging to 33 classes.
Using 71612 files for training.
Found 89514 files belonging to 33 classes.
Using 17902 files for validation.


In [4]:
do_fine_tuning = True
print("Building model with", model_handle)
model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(model_handle, trainable=do_fine_tuning),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(len(class_names),
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

Building model with https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/feature_vector/2
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              20331360  
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 33)                42273     
Total params: 20,373,633
Trainable params: 20,219,761
Non-trainable params: 153,872
_________________________________________________________________


In [5]:
class_weight = {0: 1.567858893,
                1: 1.002192296,
                2: 1.002506266,
                3: 1.000312598,
                4: 1.33388912,
                5: 1.480111008,
                6: 1.335559265,
                7: 1.333333333,
                8: 1.000625391,
                9: 1.486298189,
                10: 1.013299557,
                11: 1.000625391,
                12: 1.33611691,
                13: 1.33388912,
                14: 1.33611691,
                15: 1.334445371,
                16: 1.33388912,
                17: 1.006605851,
                18: 1.505174036,
                19: 1.436265709,
                20: 1.013299557,
                21: 1.372801373,
                22: 1.333333333,
                23: 1.335002086,
                24: 1.000312598,
                25: 1.427297056,
                26: 1.336675021,
                27: 1.000312598,
                28: 1.000312598,
                29: 1.000625391,
                30: 1.001878522,
                31: 1,
                32: 1.001564945}

In [6]:
f_step = train_size // BATCH_SIZE
lr_decayed_fn = (tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=0.03,t_mul=1,m_mul=1,first_decay_steps=f_step,alpha=0.0))
model.compile(
  optimizer=tf.keras.optimizers.SGD(learning_rate=lr_decayed_fn, momentum=0.9),
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])

In [9]:
steps_per_epoch = train_size // BATCH_SIZE
validation_steps = valid_size // BATCH_SIZE
MyCallback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3,restore_best_weights=True)
hist = model.fit(
    train_ds,
    epochs=10, steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=validation_steps,
    callbacks=[MyCallback],
    class_weight=class_weight).history

Epoch 1/10
5967/5967 [==============================] - 2518s 415ms/step - loss: 2.1077 - accuracy: 0.6261 - val_loss: 1.2952 - val_accuracy: 0.7934
Epoch 2/10
5967/5967 [==============================] - 2420s 405ms/step - loss: 1.7381 - accuracy: 0.7323 - val_loss: 1.2220 - val_accuracy: 0.8178
Epoch 3/10
5967/5967 [==============================] - 2415s 405ms/step - loss: 1.6144 - accuracy: 0.7678 - val_loss: 1.1920 - val_accuracy: 0.8293
Epoch 4/10
5967/5967 [==============================] - 2415s 405ms/step - loss: 1.5341 - accuracy: 0.7916 - val_loss: 1.1718 - val_accuracy: 0.8337
Epoch 5/10
5967/5967 [==============================] - 2414s 405ms/step - loss: 1.4737 - accuracy: 0.8071 - val_loss: 1.1537 - val_accuracy: 0.8371
Epoch 6/10
5967/5967 [==============================] - 2416s 405ms/step - loss: 1.4242 - accuracy: 0.8213 - val_loss: 1.1448 - val_accuracy: 0.8404
Epoch 7/10
5967/5967 [==============================] - 2416s 405ms/step - loss: 1.3801 - accuracy: 0.8333

In [10]:
model.save('v2s')

INFO:tensorflow:Assets written to: v2s\assets


INFO:tensorflow:Assets written to: v2s\assets


In [12]:
from tensorflow import keras
#model = keras.models.load_model('v2s')
test_dir = ("./final_test/")
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels=None,
    color_mode='rgb',
    batch_size=1,
    image_size=(384, 384),
    shuffle=False,
    validation_split=None,
    subset=None,
    crop_to_aspect_ratio=False,
)
probability_model = tf.keras.Sequential([model,tf.keras.layers.Softmax()])
def process(image):
    image = tf.cast(image/255. ,tf.float32)
    return image
test_ds = test_ds.map(process)
predictions = probability_model.predict(test_ds)

Found 22308 files belonging to 1 classes.


In [13]:
a = np.zeros(22308)
for i in range(0,22308):
    a[i] = np.argmax(predictions[i])

In [14]:
replacements = {0:'asparagus', 1:'bambooshoots', 2:'betel', 3:'broccoli', 4:'cauliflower', 5:'chinesecabbage', 6:'chinesechives', 7:'custardapple', 8:'grape', 9:'greenhouse', 10:'greenonion', 11:'kale', 12:'lemon', 13:'lettuce', 14:'litchi', 15:'longan', 16:'loofah', 17:'mango', 18:'onion', 19:'others', 20:'papaya', 21:'passionfruit', 22:'pear', 23:'pennisetum', 24:'redbeans', 25:'roseapple', 26:'sesbania', 27:'soybeans', 28:'sunhemp', 29:'sweetpotato', 30:'taro', 31:'tea', 32:'waterbamboo'}
# Use the map() function to replace the values in the array
b = np.array(list(map(lambda x: replacements.get(x, x), a)))
np.savetxt('label.csv', b,fmt='%s', delimiter=',')